Import Dependancies

In [1]:
# Dependencies
from config import (gkey)
import gmaps
import numpy as np
import pandas as pd
from ipywidgets.embed import embed_minimal_html

#import cleaned county data and county geo data
county_data = pd.read_csv("Tay_Resources/state_county_w_loc.csv")
county_geo_data = pd.read_csv("Tay_Resources/uscounties_all.csv")

#add in column for Average Life Expectancy
county_data["Life Exp Avg"]=((county_data["Difference in male life expectancy, 1985-2010 (years)"]+
                              county_data["Difference in female life expectancy, 1985-2010 (years)"])/2)

#merge dataframes together
complete_county_data = pd.merge(county_geo_data, county_data, on='Location')

In [2]:
#store all counties with negative life expectancy growth
negative_counties = complete_county_data.loc[(complete_county_data["Life Exp Avg"] < 0),:]

# turns all negative 'avg life ex' values to zero
complete_county_data.loc[(complete_county_data["Life Exp Avg"] < 0), "Life Exp Avg"] = 0

In [4]:
#view stats on average expectancy
complete_county_data['Life Exp Avg'].aggregate(['mean','median','var','std','min','max'])

mean       2.718570
median     2.700000
var        1.841566
std        1.357043
min        0.000000
max       10.700000
Name: Life Exp Avg, dtype: float64

In [5]:
# create two dataframes above mean and below mean
above_county_data = complete_county_data.loc[(complete_county_data["Life Exp Avg"] >= 2.7),:]
below_county_data = complete_county_data.loc[(complete_county_data["Life Exp Avg"] < 2.7),:]

In [6]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

## Average Growth in Life Expectancy

In [13]:
# Store 'Lat' and 'Lng' into  locations 
locations_a = above_county_data[["lat", "lng"]]
locations_b = below_county_data[["lat", "lng"]]

fig1 = gmaps.figure()

#add below average layer 
below_avg = gmaps.heatmap_layer(locations_b, dissipating=False, weights=below_county_data['Life Exp Avg'],
                                max_intensity=10,point_radius = 1, gradient = ['white','blue'])
fig1.add_layer(below_avg)

#add above average layer 
above_avg = gmaps.heatmap_layer(locations_a, dissipating=False, weights=above_county_data['Life Exp Avg'],
                                max_intensity=10, point_radius = .7)
fig1.add_layer(above_avg)


#add negative counties
neg_locs = negative_counties[['lat', 'lng']]
neg_counties = gmaps.symbol_layer(neg_locs, fill_color='white', stroke_color='white', scale=1)
fig1.add_layer(neg_counties)


embed_minimal_html('export.html', views=[fig1])

fig1

Figure(layout=FigureLayout(height='420px'))